In [2]:
# Ultralytics YOLO 🚀, GPL-3.0 license

# import hydra
import torch
from ultralytics import YOLO
from ultralytics.engine.predictor import BasePredictor
# from ultralytics import DEFAULT_CONFIG, ROOT, ops
from ultralytics.models.utils import ops
from ultralytics.utils.checks import check_imgsz
from ultralytics.utils.plotting import Annotator, colors, save_one_box

c:\Users\Dell\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
import torch
from ultralytics import YOLO
import easyocr
import cv2


reader = easyocr.Reader(['en'], gpu=True)


def perform_ocr_on_image(img,coordinates):
    img = cv2.imread(img)
    x, y, w, h = map(int, coordinates)
    cropped_img = img[y:h, x:w]

    gray_img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    results = reader.readtext(gray_img)

    text = ""
    for res in results:
        if len(results) == 1 or (len(res[1]) > 6 and res[2] > 0.2):
            text = res[1]

    return str(text)

def helmet_detect(img):
    global wearing_helmet
    wearing_helmet = True
    helmet_model = YOLO("helmet.pt")
    helmet_pred = helmet_model(img)
    list_of_helmets = list(helmet_pred[0].boxes.cls.numpy())
    if 1.0 in list_of_helmets:
        wearing_helmet = False

def detect_read_numberplate(img):
    numberplate_model = YOLO("best.pt")
    detect_plate = numberplate_model(img,save_crop = True)
    coords = list(detect_plate[0].boxes.xyxy.numpy()[0])
    text = perform_ocr_on_image(img,coords)
    return text


def run():
    img = r"C:\Users\Dell\Downloads\WhatsApp Image 2024-04-10 at 21.44.15 (1).jpeg"  # Idhar image ka path daalneka
    wearing_helmet = True
    helmet_detect(img)
    text = detect_read_numberplate(img)
    print(text)

if __name__ == "__main__":
    run()

In [3]:
import easyocr
import cv2

In [ ]:
def perform_ocr_on_image(img, coordinates):
    x, y, w, h = map(int, coordinates)
    cropped_img = img[y:h, x:w]

    gray_img = cv2.cvtColor(cropped_img, cv2.COLOR_RGB2GRAY)
    results = reader.readtext(gray_img)

    text = ""
    for res in results:
        if len(results) == 1 or (len(res[1]) > 6 and res[2] > 0.2):
            text = res[1]

    return str(text)

Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


In [5]:
numberplate_model = YOLO("best.pt")
detect_plate = numberplate_model(r"image.PNG")


image 1/1 c:\Users\Dell\Downloads\HelmetDetect\HelmetDetect\image.PNG: 544x640 1 licence, 132.0ms
Speed: 10.0ms preprocess, 132.0ms inference, 9.0ms postprocess per image at shape (1, 3, 544, 640)


In [6]:
list1 = detect_plate[0].boxes.xyxy.numpy()

In [10]:
p = list(list1[0])

In [13]:
x, y, w, h = map(int, p)

int

In [18]:
perform_ocr_on_image("image.PNG",p)

'Dutud'

In [97]:
list1

array([     59.998,      275.14,       172.7,      296.78], dtype=float32)

In [14]:
model = YOLO('helmet.pt')

In [29]:
helmetpred = model(r"C:\Users\Dell\OneDrive\Pictures\Screenshots\Screenshot 2024-04-10 200540.png")


image 1/1 C:\Users\Dell\OneDrive\Pictures\Screenshots\Screenshot 2024-04-10 200540.png: 192x416 6 helmets, 1 no_helmet, 118.8ms
Speed: 5.0ms preprocess, 118.8ms inference, 2.0ms postprocess per image at shape (1, 3, 192, 416)


In [47]:
list_of_helmets = list(helmetpred[0].boxes.cls.detach().numpy())

In [52]:
0 in list_of_helmets

True

In [9]:
class DetectionPredictor(BasePredictor):

    def get_annotator(self, img):
        return Annotator(img, line_width=self.args.line_thickness, example=str(self.model.names))

    def preprocess(self, img):
        img = torch.from_numpy(img).to(self.model.device)
        img = img.half() if self.model.fp16 else img.float()  # uint8 to fp16/32
        img /= 255  # 0 - 255 to 0.0 - 1.0
        return img

    def postprocess(self, preds, img, orig_img):
        preds = ops.non_max_suppression(preds,
                                        self.args.conf,
                                        self.args.iou,
                                        agnostic=self.args.agnostic_nms,
                                        max_det=self.args.max_det)

        for i, pred in enumerate(preds):
            shape = orig_img[i].shape if self.webcam else orig_img.shape
            pred[:, :4] = ops.scale_boxes(img.shape[2:], pred[:, :4], shape).round()

        return preds

    def write_results(self, idx, preds, batch):
        p, im, im0 = batch
        log_string = ""
        if len(im.shape) == 3:
            im = im[None]  # expand for batch dim
        self.seen += 1
        im0 = im0.copy()
        if self.webcam:  # batch_size >= 1
            log_string += f'{idx}: '
            frame = self.dataset.count
        else:
            frame = getattr(self.dataset, 'frame', 0)

        self.data_path = p
        # save_path = str(self.save_dir / p.name)  # im.jpg
        self.txt_path = str(self.save_dir / 'labels' / p.stem) + ('' if self.dataset.mode == 'image' else f'_{frame}')
        log_string += '%gx%g ' % im.shape[2:]  # print string
        self.annotator = self.get_annotator(im0)

        det = preds[idx]
        self.all_outputs.append(det)
        if len(det) == 0:
            return log_string
        for c in det[:, 5].unique():
            n = (det[:, 5] == c).sum()  # detections per class
            log_string += f"{n} {self.model.names[int(c)]}{'s' * (n > 1)}, "
        # write
        gn = torch.tensor(im0.shape)[[1, 0, 1, 0]]  # normalization gain whwh
        for *xyxy, conf, cls in reversed(det):
            if self.args.save_txt:  # Write to file
                xywh = (ops.xyxy2xywh(torch.tensor(xyxy).view(1, 4)) / gn).view(-1).tolist()  # normalized xywh
                line = (cls, *xywh, conf) if self.args.save_conf else (cls, *xywh)  # label format
                with open(f'{self.txt_path}.txt', 'a') as f:
                    f.write(('%g ' * len(line)).rstrip() % line + '\n')

            if self.args.save or self.args.save_crop or self.args.show:  # Add bbox to image
                c = int(cls)  # integer class
                label = None if self.args.hide_labels else (
                    self.model.names[c] if self.args.hide_conf else f'{self.model.names[c]} {conf:.2f}')
                
                
                text_ocr = perform_ocr_on_image(im0,xyxy)
                label = text_ocr 
                
                self.annotator.box_label(xyxy, label, color=colors(c, True))
            if self.args.save_crop:
                imc = im0.copy()
                save_one_box(xyxy,
                             imc,
                             file=self.save_dir / 'crops' / self.model.model.names[c] / f'{self.data_path.stem}.jpg',
                             BGR=True)

        return log_string

In [12]:
# def predict(cfg):
#     cfg.model = cfg.model or "yolov8n.pt" #"best.pt"  
#     cfg.imgsz = check_imgsz(cfg.imgsz, min_dim=2)  # check image size
    # cfg.source = cfg.source if cfg.source is not None else ROOT / "assets"
predictor = DetectionPredictor()
predictor()


Ultralytics YOLOv8.1.34 🚀 Python-3.11.5 torch-2.1.0+cpu CPU (11th Gen Intel Core(TM) i5-11260H 2.60GHz)


TypeError: model='None' is not a supported model format. See https://docs.ultralytics.com/modes/predict for help.

                   Format     Argument           Suffix    CPU    GPU
0                 PyTorch            -              .pt   True   True
1             TorchScript  torchscript     .torchscript   True   True
2                    ONNX         onnx            .onnx   True   True
3                OpenVINO     openvino  _openvino_model   True  False
4                TensorRT       engine          .engine  False   True
5                  CoreML       coreml       .mlpackage   True  False
6   TensorFlow SavedModel  saved_model     _saved_model   True   True
7     TensorFlow GraphDef           pb              .pb   True   True
8         TensorFlow Lite       tflite          .tflite   True  False
9     TensorFlow Edge TPU      edgetpu  _edgetpu.tflite   True  False
10          TensorFlow.js         tfjs       _web_model   True  False
11           PaddlePaddle       paddle    _pa

In [11]:
# if __name__ == "__main__":
#     predict()

TypeError: predict() missing 1 required positional argument: 'cfg'